# Time In Process
This Jupyter Notebook address the questions mentioned in the prompts for the Ashby take home test.  The questions are:
1. What is the company's Time in Process? Is that good or bad?
1. How is the company's Time in Process trending?
1. How does the company's Time in Process vary across myriad segments?
    - For example, Time in Proccess by department, location, application outcome, etc.
1. What is the average or median Time in Process for specific stages or stage groups?

Additional points to consider:
- It is best to focus results and analysis at the stage group level.
- It is common for a candidate's application to enter a given stage more than once.
- While there is no bad data, data can be removed if it appears erroneous, but documentation is required for the removal.
- Customer Success team doesn't have the ability to create their own datasets or visualization content, but can apply filters or updates to content provided for them.

## What is the company's Time in Process?

In [ ]:
import duckdb

duckdb.connect("../../database/ashby.db", read_only= True)
